# Testing C++ code in python
June 12, 2020 \
June 20, 2020 : Added test for batch of images with multi-channels

In [1]:
import numpy as np

In [2]:
xsize,ysize=128,128
# xsize,ysize=10,10
batch_size,num_channels=100,2

## Save image for C++ code

In [ ]:
# ### Save single 2D image to csv for C++ code to handle
fname='data/gen_images.npy'
a1=np.load(fname)
print(a1.shape)

a2=a1[0:batch_size*num_channels,0,:xsize,:ysize]
print(a2.shape)
np.savetxt('data/images.csv',a2.flatten(),delimiter=',',newline=',')

### Save second image
a2=a1[1*batch_size*num_channels:2*batch_size*num_channels,0,:xsize,:ysize]
print(a2.shape)
np.savetxt('data/images2.csv',a2.flatten(),delimiter=',',newline=',')

### Test for single image

In [ ]:
fname='data/images.csv'
x=np.loadtxt(fname,delimiter=',',dtype=str)[:-1].astype(np.float64).reshape(xsize,ysize)
print(x.shape)
# x

In [ ]:
arr=np.array([(i+1)*10+ 5*j for i in range(0,5) for j in range (0,5)]).reshape(5,5).T
arr

In [ ]:
y1=np.fft.fft2(arr)
# y1=np.fft.fftshift(y1)
y2=abs(y1)

In [ ]:
print(y2)

In [ ]:
y1

In [ ]:
### Read c++ file
cp_ip_file='data/op.csv'
# x=np.loadtxt(fname,delimiter=',',dtype=str)[:-1].astype(np.float64).reshape(5,5)
z_cpp=np.loadtxt(cp_ip_file,delimiter=',',dtype=str)[:-1].astype(np.float64)
print(z_cpp)

In [ ]:
### Check arrays
np.allclose(z1,z_cpp[:z1.shape[0]],rtol=1e-4,atol=1e-8)

In [ ]:
### Check manually elementwise
for i in range(z1.shape[0]):
    if (np.around(z1[i],4)!=np.around(z_cpp[i],4)): 
          print(i,z1[i],z_cpp[i])

## Multi-channel batch example

### Read input

In [3]:
fname='data/images2.csv'
x=np.loadtxt(fname,delimiter=',',dtype=str)[:-1].astype(np.float64).reshape(batch_size,num_channels,xsize,ysize)
print(x.shape)
# x

(100, 2, 128, 128)


### Compute batch spectrum

In [4]:
def f_radial_profile(data, center=(None,None)):
    ''' Module to compute radial profile of a 2D image '''
    y, x = np.indices((data.shape)) # Get a grid of x and y values
    
    if center[0]==None and center[1]==None:
        center = np.array([(x.max()-x.min())/2.0, (y.max()-y.min())/2.0]) # compute centers
        
    # get radial values of every pair of points
    r = np.sqrt((x - center[0])**2 + (y - center[1])**2)
    r = r.astype(np.int)
    
    # Compute histogram of r values
    tbin = np.bincount(r.ravel(), data.ravel())
    nr = np.bincount(r.ravel()) 
    radialprofile = tbin / nr
    
    return radialprofile

def f_compute_spectrum(arr):
    y1=np.fft.fft2(arr)
    y1=np.fft.fftshift(y1)
    y2=abs(y1)
    z1=f_radial_profile(y2)
    return(z1)
   
def f_compute_batch_spectrum(arr):
    batch_pk=np.array([f_compute_spectrum(i) for i in arr])
    return batch_pk


### Code ###
def f_image_spectrum(x):
    '''
    Data has to be in the form (batch,channel,x,y)
    '''
    print(x.shape)
    mean=[[] for i in range(num_channels)]    
    sdev=[[] for i in range(num_channels)]    

    for i in range(num_channels):
        arr=x[:,i,:,:]
#         print(i,arr.shape)
        batch_pk=f_compute_batch_spectrum(arr)
#         print(batch_pk)
        mean[i]=np.mean(batch_pk,axis=0)
        sdev[i]=np.std(batch_pk,axis=0)
    mean=np.array(mean)
    sdev=np.array(sdev)
    return mean,sdev


In [5]:
mean,sdev=f_image_spectrum(x)
print(mean.shape,sdev.shape)

(100, 2, 128, 128)
(2, 90) (2, 90)


In [6]:
# print(mean,sdev)

### Read from c++ files and compare

In [7]:
### Read c++ files
cp_ip_file='data/op_spec_mean.csv'
# x=np.loadtxt(fname,delimiter=',',dtype=str)[:-1].astype(np.float64).reshape(5,5)
z_cpp1=np.loadtxt(cp_ip_file,delimiter=',',dtype=str)[:-1].astype(np.float64).reshape(num_channels,-1)

cp_ip_file='data/op_spec_sdev.csv'
# x=np.loadtxt(fname,delimiter=',',dtype=str)[:-1].astype(np.float64).reshape(5,5)
z_cpp2=np.loadtxt(cp_ip_file,delimiter=',',dtype=str)[:-1].astype(np.float64).reshape(num_channels,-1)
# print(z_cpp1,'\n',z_cpp2)

In [8]:
z_cpp1

array([[3110.45   ,  232.301  ,  208.574  ,  179.257  ,  163.541  ,
         147.482  ,  132.39   ,  119.615  ,  112.616  ,  101.257  ,
          95.034  ,   90.4084 ,   83.3991 ,   78.0702 ,   72.7444 ,
          68.5949 ,   64.5804 ,   60.9793 ,   55.5383 ,   51.097  ,
          47.7847 ,   45.0424 ,   42.6496 ,   40.1907 ,   37.9842 ,
          35.5928 ,   33.8078 ,   32.5531 ,   31.614  ,   29.471  ,
          27.6195 ,   26.932  ,   25.9147 ,   24.3856 ,   22.7196 ,
          22.1321 ,   20.7962 ,   19.4705 ,   18.6831 ,   17.8452 ,
          17.3083 ,   16.5005 ,   15.9245 ,   15.3207 ,   14.6979 ,
          14.5066 ,   13.5859 ,   13.4838 ,   13.0238 ,   12.422  ,
          11.6107 ,   11.6651 ,   11.1717 ,   10.8709 ,   10.6746 ,
          10.4208 ,   10.581  ,   10.4803 ,    9.98672,    9.69886,
           9.4879 ,    9.39094,    9.10467,    9.11069,    8.57453,
           7.7591 ,    7.30037,    7.18052,    7.25239,    6.60582,
           6.567  ,    6.73298,    6.34804,    6

In [9]:
mean

array([[3110.45170677,  232.30093957,  208.5740333 ,  179.25690948,
         163.54105753,  147.48208321,  132.39015146,  119.61487282,
         112.61634237,  101.25726236,   95.03403846,   90.40839065,
          83.39909775,   78.07020817,   72.74441356,   68.59485977,
          64.58042604,   60.97933525,   55.53832231,   51.09702481,
          47.78465445,   45.04238529,   42.64957716,   40.1906578 ,
          37.98417557,   35.59281844,   33.80782908,   32.55306526,
          31.61396659,   29.47102245,   27.61949628,   26.93199773,
          25.91474823,   24.38558906,   22.71963417,   22.13209269,
          20.79618852,   19.4704603 ,   18.68313299,   17.84518891,
          17.30831066,   16.50052629,   15.92451878,   15.32071781,
          14.6978647 ,   14.50658415,   13.58588147,   13.48380015,
          13.0237923 ,   12.42202254,   11.61071318,   11.66509301,
          11.1716751 ,   10.87085079,   10.67456735,   10.42075577,
          10.58102541,   10.48032836,    9.98671

In [10]:
### Check arrays
print(np.allclose(mean,z_cpp1[:,:mean.shape[1]],rtol=1e-4,atol=1e-8))
print(np.allclose(sdev,z_cpp2[:,:sdev.shape[1]],rtol=1e-4,atol=1e-8))


True
True


## Conclusion:
###### July 2, 2020
Reading input + computing FFT + computing Modulus + Getting Radial profile \
Arrays from python and C++ match !!\
The C++ code is correct and ready to port to LBANN

###### July 20, 2020
Reading input + computing FFT + computing Modulus + Getting Radial profile + computing mean, sdev \
Arrays from python and C++ match !!\


####  Update
October 26, 2020 \
Adding a fix for $ fftshift $ \
Checked results for 2 different images. Success!!\

## Testing spectral loss

In [11]:
fname='data/images.csv'
x1=np.loadtxt(fname,delimiter=',',dtype=str)[:-1].astype(np.float64).reshape(batch_size,num_channels,xsize,ysize)
fname='data/images2.csv'
x2=np.loadtxt(fname,delimiter=',',dtype=str)[:-1].astype(np.float64).reshape(batch_size,num_channels,xsize,ysize)


In [12]:
mean1,sdev1=f_image_spectrum(x1)
mean2,sdev2=f_image_spectrum(x2)


(100, 2, 128, 128)
(100, 2, 128, 128)


In [13]:
mean1.shape

(2, 90)

In [14]:
k_crop=int(xsize/2) ## =64
np.log(np.mean(np.square(mean1[:,:k_crop]-mean2[:,:k_crop]))),np.log(np.mean(np.square(sdev1[:,:k_crop]-sdev2[:,:k_crop])))

(2.38995646839355, 1.682616115504929)

### Conclusion:
July 28,2020

Results matches c++ code output exactly

#### Update 
October 26, 2020
Results matches after the fix for fftshift
